### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6235,2023-11-20,Brasil Nbb,19:30,Bauru,União Corinthians,1.28,3.59,152.5,1.87,1.87,-9.5,2.03,1.70,4jjgJntb,0.781250,0.278552,0.534759,0.534759,0.059802,0.0,0.0,NaN,NaN,151.996,66.493489,0.437469,134.204,48.435833,0.360912,0.000,0.000,105.08,207.92,0.0,0.0,0.0,0.0,0.670808,0.000000,0.125118,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6236,2023-11-20,Brasil Nbb,20:00,São Paulo,Pato,1.23,4.05,157.5,1.82,1.84,-10.5,1.96,1.75,Q5v2HQAA,0.813008,0.246914,0.549451,0.543478,0.059922,0.0,0.0,NaN,NaN,119.342,35.727989,0.299375,255.322,165.082624,0.646566,0.000,0.000,146.64,203.58,0.0,0.0,0.0,0.0,0.755319,0.007728,0.080050,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6237,2023-11-20,Eua Nba,21:00,Washington Wizards,Milwaukee Bucks,4.23,1.28,242.5,1.82,2.06,8.5,2.06,1.87,MDy2HGGh,0.236407,0.781250,0.549451,0.485437,0.017657,0.2,1.0,NaN,NaN,356.252,122.596132,0.344128,180.324,31.875183,0.176766,280.884,181.370,331.65,197.12,0.0,0.0,0.0,0.0,0.757156,0.087477,0.068372,-2.95,-0.590,-5.474576,0.394435,0.4,0.005565,-2.01,-0.402,-0.696517,0.681919,0.5,-0.181919
6238,2023-11-20,Eua Nba,22:00,Chicago Bulls,Miami Heat,2.11,1.85,208.5,1.80,2.09,1.5,1.97,1.97,04z6GzWb,0.473934,0.540541,0.555556,0.478469,0.014474,0.4,0.0,NaN,NaN,236.328,115.740815,0.489747,208.826,53.304771,0.255259,193.518,217.846,234.60,167.81,0.0,0.0,0.0,0.0,0.092852,0.105430,0.000000,-1.37,-0.274,-4.051095,0.600422,0.4,-0.200422,1.05,0.210,4.047619,0.403154,0.5,0.096846
6239,2023-11-20,Eua Nba,22:00,Minnesota Timberwolves,New York Knicks,1.69,2.37,211.5,1.80,2.08,-3.5,2.02,1.92,UwuAFfo5,0.591716,0.421941,0.555556,0.480769,0.013657,0.4,0.4,NaN,NaN,223.490,77.534872,0.346928,229.458,103.189244,0.449709,202.508,258.108,169.58,172.02,0.0,0.0,0.0,0.0,0.236863,0.102057,0.035894,1.79,0.358,1.927374,0.638326,0.7,0.061674,-0.11,-0.022,-62.272727,0.435747,0.4,-0.035747
6240,2023-11-20,Eua Nba,22:00,New Orleans Pelicans,Sacramento Kings,1.95,1.85,236.5,1.85,1.95,-2.5,2.06,1.71,8UwEEE0B,0.512821,0.540541,0.540541,0.512821,0.053361,0.8,0.8,NaN,NaN,319.922,31.769854,0.099305,213.898,44.816442,0.209522,288.186,249.758,362.40,175.44,1.0,0.0,0.0,0.0,0.037216,0.037216,0.131293,0.01,0.002,475.000000,0.560106,0.5,-0.060106,3.43,0.686,1.239067,0.456416,0.5,0.043584
6241,2023-11-20,Eua Nba,22:00,San Antonio Spurs,Los Angeles Clippers,3.39,1.39,228.5,1.84,1.94,5.5,2.07,1.74,jgTIDYFH,0.294985,0.719424,0.543478,0.515464,0.014410,1.0,0.4,NaN,NaN,364.542,86.870515,0.238300,198.516,77.600779,0.390904,303.752,247.540,255.96,284.04,0.0,0.0,0.0,0.0,0.591721,0.037413,0.122491,-3.20,-0.640,-3.734375,0.387923,0.3,-0.087923,-5.00,-1.000,-0.390000,0.450770,0.2,-0.250770
6242,2023-11-20,Itália Liga A,16:30,Virtus Bologna,Brescia,1.28,3.80,161.5,1.81,1.92,-9.5,2.01,1.77,M3UJhZjH,0.781250,0.263158,0.552486,0.520833,0.044408,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,95.55,187.05,0.0,0.0,0.0,0.0,0.701539,0.041706,0.089791,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6243,2023-11-20,Alemanha Bbl,16:00,Crailsheim Merlins,Heidelberg,1.95,1.87,170.5,1.80,1.93,-1.5,2.06,1.71,nuB2luYE,0.512821,0.534759,0.555556,0.518135,0.047580,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,201.60,158.86,0.0,0.0,0.0,0.0,0.029617,0.049289,0.131293,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6244,2023-11-20,Argentina Liga A,21:10,Penarol,Insti

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,21:00,Eua Nba,Washington Wizards,Milwaukee Bucks,242.5,1.82,0.9999,Over
3,22:00,Eua Nba,Chicago Bulls,Miami Heat,208.5,1.80,1.0000,Over
4,22:00,Eua Nba,Minnesota Timberwolves,New York Knicks,211.5,1.80,1.0000,Over
22,13:30,Eua Ncaa,Siena,Central Michigan,140.5,2.00,0.8020,Over
